### STEPUP

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


In [2]:
pip install torch

### THE PROJECT 



In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.optim as optim
from IPython.display import HTML
import cv2
from tqdm import tqdm


from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [5]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

env = gym.make("SpaceInvadersDeterministic-v4")
num_actions = env.action_space
print("num_actions : ", num_actions)

num_actions :  Discrete(6)


In [0]:
#### HYPERPARAMETERS ####

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPISODES = 500
EXEPRIENCE_SIZE = 20000
THRESHOLD =  5
GAMMA = 0.95
RANDOM_ACTION_CHANCE = 1
MIN_RANDOM_ACTION_CHANCE = 0.05
RADR = 0.9
REPLACE = True
REPLACE_INTERVAL = 1000


In [0]:
#### DATALOADER ####




In [0]:
#### PREPROCESSING ####

def preprocess(x):
  x = np.mean( x[15:200, 30:125], axis=2 )
  return x


In [8]:
#### MODEL ####

class Network(nn.Module):

    def __init__ (self):

        super(Network, self).__init__()
        '''
        self.conv_1 = nn.Conv2d(3, 32, kernel_size=5, stride=2)
        #nn.init.xavier_uniform_(self.conv_1.weight, gain=np.s1qrt(2))
        #self.bn_1 = nn.BatchNorm2d(16)
        
        self.conv_2 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        #nn.init.xavier_uniform_(self.conv_2.weight, gain=np.sqrt(2))
        #self.bn_2 = nn.BatchNorm2d(32)
        
        self.conv_3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        #nn.init.xavier_uniform_(self.conv_3.weight, gain=np.sqrt(2))
        #self.bn_3 = nn.BatchNorm2d(32)
        '''

        self.conv_1 = nn.Conv2d(3, 32, 8, stride=4, padding=1)
        self.conv_2 = nn.Conv2d(32, 64, 4, stride=2)
        self.conv_3 = nn.Conv2d(64, 128, 3)

        self.fc_1 = nn.Linear(128*8*19, 512)
        #nn.init.xavier_uniform_(self.fc_1.weight, gain=np.sqrt(2))
        
        self.fc_2 = nn.Linear(512, 6)
        #nn.init.xavier_uniform_(self.fc_2.weight, gain=np.sqrt(2))

    def forward(self, x, predict_action=False):
        
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = F.relu(self.conv_3(x))
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = F.relu(self.fc_1(x))
        
        x = self.fc_2(x)
        if predict_action:
          x = torch.argmax(x,dim=1)
        return x

model = Network().cuda()
criterion = F.smooth_l1_loss

test, test1 , test2 = torch.rand( (BATCH_SIZE,3,185,95)).cuda(), torch.rand(BATCH_SIZE), torch.rand(BATCH_SIZE)
criterion(test1, test2), model(test, predict_action = True).shape, model(test).shape


(tensor(0.0525), torch.Size([32]), torch.Size([32, 6]))

In [9]:
### Init Memory ###

env = gym.make("SpaceInvadersDeterministic-v4")

Experience = []

while len(Experience)<EXEPRIENCE_SIZE:

  action = 0
  frame_num = 0
  frames = [preprocess(env.reset())]
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  done_val = 1
  print("Done")

  while True:
    
    curr_state = np.array( frames )

    action = int(np.random.randint(0,6,1))

    observation, reward , done , _ = env.step(action)
    frames.pop(0) 
    frames.append( preprocess( observation ))    
    next_state = np.array( frames )

    if done:
      reward = -1000
      done_val = 0

    if len(Experience) < EXEPRIENCE_SIZE:
      Experience.append( [ curr_state, action, next_state, reward, done_val] ) 

    else:
      Experience.pop(0)
      Experience.append( [ curr_state, action, next_state, reward, done_val] )

    if done:
      break

env.close()
len(Experience), len(Experience[0]), Experience[0][0].shape

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


(20000, 5, (3, 185, 95))

In [0]:
def train(replace_counter):

  if REPLACE and replace_counter%REPLACE_INTERVAL==0:
    # print( "\n Replaced \n")
    Q_target.load_state_dict( Q_model.state_dict() )
    replace_counter = 0

  optimizer.zero_grad()

  idx = np.random.randint( 0 , len(Experience) - BATCH_SIZE-1)
  Training_Experiences = np.array(Experience[idx : idx + BATCH_SIZE])

  curr_frames = np.array( list (Training_Experiences[:,0]))
  predicted_q_values = Q_model(torch.FloatTensor(curr_frames).cuda())
  actions = torch.LongTensor(np.array(list(Training_Experiences[:,1]))).cuda().unsqueeze(1)
  action_predicted_q_values = torch.gather(predicted_q_values,1,actions).squeeze()
  
  next_frames = np.array(list(Training_Experiences[:,2]))
  with torch.no_grad():
    max_q_values, _ = Q_target(torch.FloatTensor(next_frames).cuda()).max( dim=1 )
  
  rewards = torch.FloatTensor(np.array(list(Training_Experiences[:,3]))).cuda()
  
  done_vals = torch.Tensor(np.array(list(Training_Experiences[:,4]))).cuda()

  utilities = rewards + max_q_values*done_vals*GAMMA

  loss = criterion( action_predicted_q_values, utilities)

  loss.backward()

  optimizer.step()


In [0]:
def evaluate():
  total_reward = 0
  action = 0
  frame_num = 0
  frames = [preprocess(env.reset())]
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  done_val = 1

  while True:

    curr_state = np.array( frames )

    cuda_state = torch.Tensor(np.expand_dims(curr_state,axis=0)).cuda()
    with torch.no_grad():
      action = Q_model(cuda_state, predict_action=True).item()
    
    observation, reward , done , info = env.step(action)
    total_reward+=reward

    frames.pop(0) 
    frames.append( preprocess( observation ))    
    
    if done:
      break

  return total_reward


In [12]:
%%time

### TRAINING LOOP ###

env = gym.make("SpaceInvadersDeterministic-v4")

Q_model = Network().cuda()
Q_target = Network().cuda()

optimizer = torch.optim.Adam(Q_model.parameters(), lr = LEARNING_RATE)
replace_counter = 0

max_r = 0

for i in range(200):

  total_reward = 0
  action = 0
  frame_num = 0
  frames = [preprocess(env.reset())]
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  observation, _ , _ , _ = env.step(action) 
  frames.append( preprocess( observation ))      
  done_val = 1

  while True:

    curr_state = np.array( frames )

    if np.random.rand() < RANDOM_ACTION_CHANCE:
      action = int(np.random.randint(0,6,1))
    else:
      cuda_state = torch.Tensor(np.expand_dims(curr_state,axis=0)).cuda()
      with torch.no_grad():
        action = Q_model(cuda_state, predict_action=True).item()
      
    observation, reward , done , info = env.step(action)
    total_reward+=reward

    frames.pop(0) 
    frames.append( preprocess( observation ))    
    
    next_state = np.array( frames )

    if done:
      Experience.pop(0)
      Experience.append( [ curr_state, action, next_state, -1000, 0] )
      break

    Experience.pop(0)
    Experience.append( [ curr_state, action, next_state, reward, done_val] )

    train(replace_counter)
    replace_counter+=1
    
    train(replace_counter)
    replace_counter+=1
    
    train(replace_counter)
    replace_counter+=1

    train(replace_counter)
    replace_counter+=1
    
    train(replace_counter)
    replace_counter+=1
    
  if RANDOM_ACTION_CHANCE>MIN_RANDOM_ACTION_CHANCE and i>THRESHOLD:
    RANDOM_ACTION_CHANCE = RANDOM_ACTION_CHANCE*RADR

  r = 0
  if RANDOM_ACTION_CHANCE<1:
    r = evaluate()
    if r>max_r:
      max_r=r
      torch.save(Q_model.state_dict(), "Best_model.pwf")
      print("Saved")
        

  print( " Game : ", i , " Epsilon : ", RANDOM_ACTION_CHANCE,  "Training Score : ", total_reward, "Evaluation Score : ", r)


env.close()

 Game :  0  Epsilon :  1 Training Score :  45.0 Evaluation Score :  0
 Game :  1  Epsilon :  1 Training Score :  135.0 Evaluation Score :  0
 Game :  2  Epsilon :  1 Training Score :  185.0 Evaluation Score :  0
 Game :  3  Epsilon :  1 Training Score :  60.0 Evaluation Score :  0
 Game :  4  Epsilon :  1 Training Score :  270.0 Evaluation Score :  0
 Game :  5  Epsilon :  1 Training Score :  375.0 Evaluation Score :  0
Saved
 Game :  6  Epsilon :  0.9 Training Score :  210.0 Evaluation Score :  65.0
 Game :  7  Epsilon :  0.81 Training Score :  120.0 Evaluation Score :  10.0
Saved
 Game :  8  Epsilon :  0.7290000000000001 Training Score :  115.0 Evaluation Score :  180.0
Saved
 Game :  9  Epsilon :  0.6561000000000001 Training Score :  20.0 Evaluation Score :  305.0
 Game :  10  Epsilon :  0.5904900000000002 Training Score :  345.0 Evaluation Score :  50.0
 Game :  11  Epsilon :  0.5314410000000002 Training Score :  320.0 Evaluation Score :  15.0
 Game :  12  Epsilon :  0.478296900000

KeyboardInterrupt: ignored

In [14]:
Q_model.load_state_dict( torch.load( "Best_model.pwf"))
for i in range(10):
  print(evaluate())

725.0
725.0
725.0
725.0
725.0
725.0
725.0
725.0
725.0
725.0


In [0]:
np.random.rand()

In [15]:

env = wrap_env(gym.make("SpaceInvadersDeterministic-v4")) 
observation = env.reset()
evaluate()

# action = 1
# r_l = 0


# env.reset()
# r_l = 0
# f = 0
# while True:  

#   action = env.action_space.sample() 
#   # action = 0

#   observation, reward, done, info = env.step(action) 

#   r_l+=reward

#   f+=1
#   if done:
#     print(r_l)
#     break

env.close()
show_video()
# print("After Video", f)

In [0]:
plt.imshow(a)

In [0]:
img = torch.ones( (64,64,3))
add = torch.tensor( [[img, 1,img, 0 ]])
add2 = torch.tensor( [[img, 2,img, 0 ]])
add3 = torch.tensor( [[img, 2,img, 0 ]])
add.shape
e = torch.empty( (0,4))
e = torch.append( e, add, axis=0)
e = torch.append( e, add2, axis=0)
e = torch.append( e, add3, axis=0)
print(e.shape)
e = np.delete(e,0,axis=0)
e[0,1]